<a href="https://colab.research.google.com/github/olaviinha/SloppyButchery/blob/main/sloppyButcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Sloppy Butcher <font color="#999" size="3">v 0.2.0<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="4">Sloppy Butchery @</font> <a href="https://github.com/olaviinha/SloppyButchery" target="_blank"><font color="#999" size="4">Github</font></a><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="3" color="#999"><a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font></font>

Sloppy Butcher is an audio power-chopper and randomizer. It takes a directory of audio files, chops it up, shuffles it, and frankensteins it into a single audio file.

###Quick start
1.   Create a directory in your Google Drive called `sloppybutcher`
2.   Drop a few longish audio files in it (and wait until Drive sync complete).
3.   Hit <i>Runtime > Run all</i> from the menu.

Sloppy Butcher is powered by Librosa, SoX and FFmpeg.

<hr size="1" color="#666"/>

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.
import os
from google.colab import output
force_setup = False
force_format = False

apt_packages = 'sox'
pip_packages = 'pysoundfile'
pip_packages_beatslice = 'spleeter essentia ffmpeg-python'
pip_packages_wordslice = 'deepspeech-gpu'

show_info = False
extra_verbose_performance = False

# Setup
if not 'setup_done' in globals() or force_setup == True:
  import sys
  hosted_runtime = 'google.colab' in sys.modules

  if hosted_runtime == True:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = "/content/drive/My Drive/"
    dir_tmp = "/content/tmp/"
    !mkdir {dir_tmp}
    !gsutil -q -m cp -R gs://olaviinha/slicer {dir_tmp}
    cleanup = False
  else:
    project_home = !pwd
    project_home = project_home[0]
    print('Running locally at', project_home, '– skipping Drive mount.')
    drive_root = project_home
    dir_tmp = project_home+"/tmp/"
    !mkdir {dir_tmp}
    cleanup = True

  !apt-get install sox
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
  import import_ipynb, soundfile
  from inhagcutils import *
  setup_done = True

class c:
  title = '\033[96m'
  ok = '\033[92m'
  okb = '\033[94m'
  warn = '\033[93m'
  fail = '\033[91m'
  endc = '\033[0m'
  bold = '\033[1m'
  u = '\033[4m'

def op(typex, msg):
  print(typex+msg+c.endc)

np.seterr(divide = 'ignore')

# Format
if not 'last_source_files' in globals() or force_format == True:
  last_source_files = []
if not 'last_conversion' in globals() or force_format == True:
  last_conversion = []
if not 'last_silence_amount' in globals() or force_format == True:
  last_silence_amount = 0
if not 'last_slices_per_beat' in globals() or force_format == True:
  last_slices_per_beat = []
if not 'last_bpm' in globals() or force_format == True:
  last_bpm = 0
if not 'last_reverse' in globals() or force_format == True:
  last_reverse = False
if not 'last_per_source_length' in globals() or force_format == True:
  last_per_source_length = 0
if not 'last_durations' in globals() or force_format == True:
  last_durations = 0











# ----------------------------------------------------------------------------------
# --- Functions
# ----------------------------------------------------------------------------------

def separate_drums(audio_track):
  warnings.filterwarnings('ignore')
  separator = Separator('/content/cfg.json')
  audio_loader = get_default_audio_adapter()
  drum_track = separator.separate(audio_track.T)['drums']
  return librosa.to_mono(drum_track.T)

def separate_vocals(audio_track):
  warnings.filterwarnings('ignore')
  separator = Separator('/content/cfg.json')
  audio_loader = get_default_audio_adapter()
  vocal_track = separator.separate(audio_track.T)['vocals']
  return librosa.to_mono(vocal_track.T)


def get_beats(audio, sr=global_sr):
  # You may use any library or tool for initial beat tracking here
  # for as long as it returns beat starting positions in seconds.
  bt = BeatTrackerMultiFeature()
  beats, _ = bt(audio)
  return beats

def get_differences(blist, round=0):
  x = list(np.array(blist.tolist()).flatten())
  xdiff = [x[n]-x[n-1] for n in range(1,len(x))]
  if round > 0:
    rounded = [ '%.2f' % el for el in xdiff ]
    return rounded
  else:
    return xdiff

def most_frequent(list):
  freq = max(set(list), key = list.count)
  return freq

def filter_durations(durations, range, decs):
  filtered_durations = []
  for duration in durations:
    duration = float(duration)
    range = float(range)
    if round(duration, decs) == range:
      filtered_durations.append(duration)
  return filtered_durations

def get_tail_peaks(audio, nudgeTime=False, wat=''):
  dur = librosa.get_duration(audio, sr=sr)
  nudged = False
  minPos = 0.96
  minPeakDis = 0.001
  th = 0.4
  pos, amp = PeakDetection(threshold=th, minPeakDistance=minPeakDis, minPosition=minPos)(audio.astype(np.float32))
  if len(pos) > 0:
    # This slice needs handling
    for i, peak in enumerate(pos):
      if amp[i] > th and nudged == False:
        nudgePeak = peak
        nudged = True
  if nudgeTime == True and nudged == True:
    return nudgePeak
  elif nudgeTime == True and nudged == False:
    return 0
  else:
    if wat == 'amp':
      return amp
    else:
      return dur-(pos*dur)

def nudge(beats, timing_track, anal_duration, real_duration, timetravel, return_type='time', starting_beat=0, sr=global_sr):
  beat_positions = []
  a = 0
  for i, beat in enumerate(beats):
    if i > starting_beat:
      # Get slice
      s_time = beat+timetravel
      e_time = anal_duration-a
      s_sample = librosa.time_to_samples(s_time, sr=sr)
      e_sample = librosa.time_to_samples(s_time+e_time, sr=sr)
      beat_timing = timing_track[s_sample:e_sample]

      # Nudge
      nudge_peak = get_tail_peaks(beat_timing, True)
      if nudge_peak > 0:
        ns_time = s_time-(anal_duration-anal_duration*nudge_peak)-a
        ns_sample = librosa.time_to_samples(ns_time, sr=sr)
      else:
        ns_time = s_time
        ns_sample = s_sample
      ns_sample = librosa.time_to_samples(ns_time-timetravel, sr=sr)
      ne_sample = librosa.time_to_samples(ns_time+(real_duration-a), sr=sr)

      # Return start:end of each beat in either 'samples' or in 'seconds'
      if return_type == 'samples':
        beat_positions.append([ns_sample, ne_sample])
      else:
        beat_positions.append([ns_time, ns_time+(real_duration-a)])
  return beat_positions

def time_stretch_audio(audio, to_length, sr=global_sr):
  dur = librosa.get_duration(audio, sr=sr)
  #librosa.effects.time_stretch(y, dur/to_length)
  return np.array([librosa.effects.time_stretch(channel, dur/to_length) for channel in split_channels(audio)])
  
def process_beat(audio, to_length, fx=[], sr=global_sr):
  stretched = time_stretch_audio(audio, to_length)
  fxd = fade_audio(apply_fx(stretched, to_length, fx))
  return fxd

def get_mode(lst):
  d = {}
  for a in lst:
    if not a in d:
      d[a]=1
    else:
      d[a]+=1
  return [k for k,v in d.items() if v==max(d.values())]

def convert(file_list, output_dir, sr=global_sr):
  for i, audiofile in enumerate(file_list):
    #print(path_leaf(audiofile), '...', end=' ')
    output = output_dir+slug(path_leaf(basename(audiofile)))+'.wav'
    filter = "pan=stereo|c0=c0|c1=c0"
    if reverse == True:
      filter = filter+", areverse"
    if normalize == True:
      filter = filter+", dynaudnorm=p=1/sqrt(2):m=100:s=12:g=15"
    !ffmpeg {ffmpeg_q} -y -i "{audiofile}" -c:a pcm_s16le -ar {sr} -ac 2 -af "{filter}" "{output}"
    #print('Done.')

def clip_sources(file_list, output_dir, duration, slice_duration, sr=global_sr):
  for i, audiofile in enumerate(file_list):
    #print('process', audiofile)
    #print('clip to', duration)
    audio_data, sr = librosa.load(audiofile, sr=sr, mono=False)
    a_duration = librosa.get_duration(audio_data, sr=sr)
    if a_duration > slice_duration*2:
      a_duration = a_duration/4 * random.randrange(2, 3)
    start = librosa.time_to_samples(a_duration, sr=sr)
    end = librosa.time_to_samples(a_duration+duration+sr, sr=sr)
    output = output_dir+path_leaf(audiofile)
    save(audio_data[:, start:end], output)
    audio_data = None

def get_duration(dir, sr=global_sr):
  files = list_audio(dir)
  duration = 0
  for file in files:
    duration += librosa.get_duration(filename=file, sr=sr)
  return duration

def slice_to_frames(audio_data, slice_duration, fade_in=global_fade, fade_out=global_fade, fx=[], sr=global_sr):
  #print('slice to frames, slice length', slice)
  a_duration = librosa.get_duration(audio_data, sr=sr)
  # print('a_duration', a_duration)
  clips = math.ceil(a_duration/slice_duration)
  # print('clips', clips)
  frames = []
  #print('total clips:', clips)
  # print('slice to', clips, end='')
  #print('fx in slice_to_frames', fx)
  for i in range(clips-1):
    #print('clip#', i)
    if i > 0 and i < clips:
      start = i*slice_duration
      #print('slice_to_frame clip_audio loop')
      audio_clip = clip_audio(audio_data, start, slice_duration, fx)
      frames.append( audio_clip ) #fade_audio(audio_clip, fade_in, fade_out) )
      #print('.', end='')
  show_mem()
  # print('done.')
  return frames

def apply_fx(audio_data, duration, fx=[]):
  # tremolo
  #print('fx in apply_fx', fx)
  if fx[0] == True:
    xtremolo = duration/2
    #print('duration', duration)
    #print('apply tremolo', xtremolo)
    audio_data = fade_audio(audio_data, xtremolo, xtremolo)
  #release
  elif fx[1] > 0:
    xrelease = fx[1]/100*duration
    audio_data = fade_audio(audio_data, global_fade, xrelease)
  # autotune
  if fx[2] != "None":
    t = math.ceil(10*duration)
    if t < 2:
      t = 3
    audio_data = autotune_audio(audio_data, note=fx[2], t=t)
    #audio_data = fade_audio(tuned)
  #pitch
  elif fx[3] != 0:
    #audio_data = fade_audio(pitch(audio_data, fx[3]))
    audio_data = pitch(audio_data, fx[3])
  if fx[4] == True:
    xfade = duration/3
    audio_data = fade_audio(audio_data, xfade, xfade)
  return audio_data

def clip_audio(audio_data, start, duration, fx=[], oneshots=False, sr=global_sr):
  global global_fade
  xstart = librosa.time_to_samples(start, sr=sr)
  xduration = librosa.time_to_samples(start+duration, sr=sr)
  #print('clip audio to duration', duration)
  #audio_data = fade_audio(audio_data[:, xstart:xduration]) 
  audio_data = audio_data[:, xstart:xduration]
  if len(fx) > 0:
    audio_data = apply_fx(audio_data, duration, fx)
  if fx[0] == False and fx[1] == 0:
    audio_data = fade_audio(audio_data) 
  
  show_mem()
  #print(audio_data.shape)
  return audio_data

def fade_audio(audio_data, fade_in=global_fade, fade_out=global_fade, sr=global_sr):
  a_duration = librosa.get_duration(audio_data, sr=sr)
  #print('in ', fade_in )
  #print('out', fade_out )
  #waveform(audio_data)
  if fade_in > 0:
    fade_in_to = librosa.time_to_samples(fade_in, sr=sr)
    in_y = audio_data[:, 0:fade_in_to]
    fade_ins = []
    for channel in in_y:
      fade = [ i/len(channel)*smp for i, smp in enumerate(channel) ]
      fade_ins.append(fade)
    fade_ins = np.array(fade_ins)
    tail_start = fade_in_to+1  
    tail = audio_data[:, tail_start:]
    audio_data = np.concatenate([fade_ins, tail], axis=1)
    #waveform(audio_data)
  if fade_out > 0:
    fade_out_start = librosa.time_to_samples(a_duration-fade_out, sr=sr)
    out_y = audio_data[:, fade_out_start:]
    fade_outs = []
    for channel in out_y:
      fade = [ smp-(i/len(channel)*smp) for i, smp in enumerate(channel) ]
      fade_outs.append(fade)
    fade_outs = np.array(fade_outs)
    head_start = fade_out_start-1
    head = audio_data[:, :head_start]
    audio_data = np.concatenate([head, fade_outs], axis=1)
    #waveform(audio_data)
  return audio_data

def split_channels(audio_data):
  return audio_data[0], audio_data[1]

def merge_channels(left_data, right_data):
  return np.array([left_data, right_data])

def detect_pitch(audio_data, t, sr=global_sr):
  pitches, magnitudes = librosa.core.piptrack(y=audio_data, sr=sr, fmin=50, fmax=900)
  # print(pitches)
  index = magnitudes[:, t].argmax()
  pitch = pitches[index, t]
  # print('detect_pitch pitch:', pitch)
  return pitch

def pitch(audio_data, semitones, sr=global_sr):
  pitched = np.array([librosa.effects.pitch_shift(channel, sr=sr, n_steps=semitones, bins_per_octave=12) for channel in split_channels(audio_data)])
  audio_data = None
  return pitched

def autotune_audio(audio_data, note='C', sr=global_sr, t=10):
  target_note = librosa.note_to_midi(note)
  # print('note', note)
  mono_audio = librosa.to_mono(audio_data)
  pitch = detect_pitch(mono_audio, t, sr=sr)
  # print('pitch hz', pitch)
  source_note = round(librosa.hz_to_midi(pitch))
  # print('hz_to_midi', midi)
  if source_note > 0:
    diff = round(target_note-source_note)
    oct = 12 if diff > 0 else -12
    octs = math.floor(diff/oct)
    if octs > 0:
      diff = diff-octs*oct
    elif octs < 0:
      diff = diff+octs*oct

    if diff < -6:
      if octs < 0:
        diff = octs*oct-diff
      else:
        diff = oct-diff
    elif diff > 6:
      if octs > 0:
        diff = octs*oct+diff
      else:
        diff = oct-diff 
    tuned = np.array([librosa.effects.pitch_shift(channel, sr=sr, n_steps=diff, bins_per_octave=12) for channel in split_channels(audio_data)])
  else:
    tuned = audio_data
  mono_audio = None
  audio_data = None
  return tuned

def generate_silence(duration, sr=global_sr):
  content = [0]*librosa.time_to_samples(duration, sr=sr)
  silence = np.array([content, content], dtype=np.float32)
  return silence

#--- saving + other -------------------------------------------------------------------------

def save(audio_data, save_as='frank', sr=global_sr):
  if save_as=='frank':
    global bpm
    timestamp = datetime.datetime.today().strftime('%Y%m%d-%H%M%S')
    save_as = save_as+'_'+rnd_str(4)+'_'+timestamp+'__'+bpm+'bpm.wav'
  soundfile.write(save_as, audio_data.T, sr)

def test_audio(audio_data):
  global dir_tmp
  if not isinstance(audio_data, (np.ndarray, np.generic)):
    global global_sr
    audio_data, sr = librosa.load(audio_data, mono=False, sr=global_sr)
  out = dir_tmp+'test_'+rnd_str(8)
  save(audio_data, out+'.wav')
  !ffmpeg {ffmpeg_q} -i {out}.wav {mp3_192} {out}.mp3
  audio_player(out+'.mp3')

def show_mem():
  global extra_verbose_performance
  if extra_verbose_performance is True:
    print('mem:', psutil.virtual_memory().percent, '/', psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)





output.clear()
op(c.ok, 'Setup finished.')


In [ ]:
#@title #Butcher

#@markdown ##Audio source material
input_dir = "sloppybutcher" #@param {type:"string"}
#handle_silence = "None" #@param ["None", "Trim_beginning_and_end", "Remove_everywhere"]
handle_silence = "None"

#@markdown <br>

#@markdown ##Saving
#@markdown <small>Checking `save_to_drive` will save every generated output file to your Drive. Uncheck if you just want to play around and listen to the previews before getting serious. Mounting Drive is still required for source audio.</small>
save_to_drive = False #@param {type:"boolean"}
output_dir = "" #@param {type:"string"}

#@markdown <br>

#@markdown ##Basics
bpm = 120 #@param {type:"slider", min:60, max:200, step:1}
slices_per_beat = 2 #@param {type:"slider", min:1, max:4}
vol = 95 #@param {type:"slider", min:1, max:100}
target_duration = 15 #@param {type:"slider", step:5, min:10, max:120}

#@markdown <br>

#@markdown ##Fancy
slice_to_beats = False #@param {type:"boolean"}
pitch_semitone = 0 #@param {type:"slider", min:-24, max:24, step:1}
autotune = "None" #@param ["None", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
reverb = 25 #@param {type:"slider", min:0, max:100, step:1}
reverse = False #@param {type:"boolean"}
normalize = False #@param {type:"boolean"}
dry_distort = False #@param {type:"boolean"}
release = 99 #@param {type:"slider", min:0, max:99}
#@markdown <small>`silence_amount` is the probability of any given beat being silence instead.</small>
silence_amount = 0 #@param {type:"slider", min:0, max:99}
#@markdown <small>`tremolo` will override `release`.</small>
tremolo = False #@param {type:"boolean"}
#@markdown <small>`crossfade` will override `release` and `tremolo`.</small>
crossfade = False #@param {type:"boolean"}

op(c.title, 'Starting...\n')

# Dirs
if hosted_runtime == True:
  input_dir = drive_root+fix_path(input_dir, True)
else:
  output_dir = project_home+"/Generated-output-WAVs/"
  !mkdir {output_dir}
  save_to_drive = True

# Clone material for Butcher
material = dir_tmp+'source-material/'
converted = dir_tmp+'converted/'
converted_clipped = dir_tmp+'converted-clipped/'
dir_slices = dir_tmp+'slices/'
dir_kicks = dir_slices+'perc-kicks/'
dir_snares = dir_slices+'perc-snares/'
dir_other = dir_slices+'perc-other/'

dirs = [material, converted, converted_clipped, dir_slices]
if output_dir != '':
  dirs.append(output_dir);
create_dirs(dirs)

source_files = list_audio(input_dir)
if source_files != last_source_files:
  if len(list_audio(material)) > 0:
    !rm {material}*
  for source_file in source_files:
    shutil.copy(source_file, material)
#copy_tree(input_dir, material)

if crossfade == True:
  slices_per_beat = slices_per_beat / 2
  approx_duration = approx_duration * 2
  trackDuration = approx_duration

# Adjustments
fade = 0.003
fade_in = fade
fade_out = fade
minute = 60
global_bpm = bpm
vol = vol/100
reverb_amount = reverb
reverb_damping = 100-reverb
pitch = pitch_semitone*100
approx_duration = target_duration
trackData = [None]
oneshots = False
silence_identifier = 'corpus_silentium'
prefix = "frankenstein"
sr = 44100
recycle_material = False

if autotune != "None":
  autotune = autotune+str(3)
  
sox_q = '-q'

global_sr = sr
global_fade = fade

# Conflict warnings
def conflictWarn(reset=False):
  global crossfade, tremolo, release, autotune, pitch_semitone
  msgtype = c.warn
  errors = False
  if crossfade == True and release > 0:
    op(msgtype, '\nWARN! Crossfade cancels release, which you have set to '+str(release)+'.')
    errors = True
    if reset == True:
      release = 0
  if crossfade == True and tremolo == True:
    op(msgtype, '\nWARN! Crossfade cancels tremolo.')
    errors = True
    if reset == True:
      tremolo = False
  if crossfade == False and tremolo == True and release > 0:
    op(msgtype, '\nWARN! Tremolo cancels release, which you have set to '+str(release)+'.')
    errors = True
    if reset == True:
      release = 0
  if autotune != "None" and pitch_semitone != 0:
    op(msgtype, '\nWARN! Autotune cancels pitch_semitone, which you have set to '+pitch_semitone+'.')
    errors = True
    if reset == True:
      pitch_semitone = 0
  if errors == False:
    if reset == True:
      print('Settings................................ OK')
    else:
      op(c.ok, 'Seems fine.')

#conflictWarn()



# ----------------------------------------------------------------------------------
# --- Beat slicing prerequisities
# ----------------------------------------------------------------------------------
if (not 'beatslice_setup_done' in globals() or force_setup == True) and slice_to_beats == True:
  print('\nSetup required Neural Networks...')
  !pip -q install {pip_packages_beatslice}
  !gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json
  import sys, warnings
  import essentia
  import essentia.standard
  import essentia.streaming
  from statistics import mode
  from essentia import Pool, array
  from essentia.standard import *
  from spleeter.separator import Separator
  from spleeter.audio.adapter import get_default_audio_adapter
  beatslice_setup_done = True

  print('Done.\n')





# ----------------------------------------------------------------------------------
# --- Basics
# ----------------------------------------------------------------------------------


bpm = global_bpm
total_slices = math.ceil((bpm/minute)*slices_per_beat*approx_duration)
trackDuration = approx_duration
pretty_trackDuration = str(datetime.timedelta(seconds=trackDuration))
slice_duration = minute/bpm/slices_per_beat

beat = minute/bpm
materialDuration = get_duration(material)
pretty_materialDuration = str(datetime.timedelta(seconds=materialDuration))

sources = glob(material+"*")
sources.sort()
per_source_length = math.ceil(trackDuration/len(sources) + 1)

if per_source_length < slice_duration:
  per_source_length = slice_duration
slices_per_source = math.ceil(total_slices / len(sources))

if show_info == True:
  op(c.title, '\n> Information')
  print('Input:..................................', input_dir)
  print('Source files:...........................', len(sources))
  print('Duration:...............................', pretty_materialDuration)

  if trackDuration > materialDuration:
    op(c.fail, 'Track target duration:.................. '+pretty_trackDuration)
  else:
    print('Track target duration:..................', pretty_trackDuration)

  print('Slices required:........................', total_slices)

  print('Required per source:....................', str(datetime.timedelta(seconds=per_source_length)))
  print('Slices per source:......................', slices_per_source)
  #print(slice * slices_per_source)
  print('BPM:....................................', bpm)
  print('Slices per beat.........................', slices_per_beat)
  print('Beat interval:..........................', beat)
  print('Slice duration:.........................', slice_duration)

  if slice_to_beats == True:
    print('Slice to beats:.........................', slice_to_beats)

#print(f"{bcolors.WARNING}Warning: No active frommets remain. Continue?{bcolors.ENDC}")

conflictWarn(reset=True)

if trackDuration > materialDuration:
  recycle_times = math.ceil(trackDuration/materialDuration)
  op(c.fail, '\nWARN! Not enough material for target duration. Material will be recycled '+str(recycle_times)+' times to meet target duration.')
  recycle_material = True























# ----------------------------------------------------------------------------------
# --- Butcher
# ----------------------------------------------------------------------------------

mode = 'rosa'

force_conversion = True
force_rough_chops = True
force_silence_generation = True
force_slice = True

op(c.title, '\nIn Progress:')

def prg(task):
  print('\r', task, end=' ')

# Convert
if sources != last_conversion or last_reverse != reverse or force_conversion == True:
  prg('Convert...')
  !rm {converted}*
  convert(sources, converted)
  converted_sources = list_audio(converted)
  process_dir = converted
  last_conversion = sources
  last_reverse = reverse

  # Rough cut
  if (((per_source_length != last_per_source_length or trackDuration+materialDuration != last_durations) and recycle_material == False) or force_rough_chops == True) and slice_to_beats == True:
    prg('Rough chops...')
    !rm {converted_clipped}*
    converted_sources = list_audio(converted)
    converted_sources.sort()
    clip_sources(converted_sources, converted_clipped, per_source_length, slice_duration)
    converted_sources = list_audio(converted_clipped)
    process_dir = converted_clipped
    last_per_source_length = per_source_length
    last_durations = trackDuration+materialDuration

prg('Recycling...')
if recycle_material == True:
  converted_concat = []
  for i in range(recycle_times):
    converted_concat.extend(converted_sources[:])
  converted_sources = converted_concat

# Slice up
if slices_per_beat != last_slices_per_beat or bpm != last_bpm or force_slice == True:
  prg('Chopping...')
  fx = [tremolo, release, autotune, pitch_semitone, crossfade]

  print(fx)

  sl_slices = dir_slices+str(slices_per_beat)+'/'
  create_dirs([sl_slices])
  #slice_time = beat/slices_per_beat
  source_slice_list = []
  all = []

  for x, source in enumerate(converted_sources):
    y, sr = librosa.load(source, sr=sr, mono=False)
    show_mem()

    if slice_to_beats == True:
      prg('Slicing to beats...')

      # Beat slicery
      timing_track = separate_drums(y)
      beats = get_beats(timing_track, sr)

      # Add precision
      decs = 2
      precise_durations = get_differences(beats, 0)
      compressed_durations = get_differences(beats, decs)
      duration_range = most_frequent(compressed_durations)
      filtered_durations = filter_durations(precise_durations, duration_range, decs)
      xs_beat = min(filtered_durations)
      m_beat = get_mode(filtered_durations)
      xl_beat = max(filtered_durations)+0.02
      nudged_beats = nudge(beats, timing_track, xl_beat, xs_beat, 0, 'samples')

      # Timestretch
      prg('Time-stretching...')
      for i, beat in enumerate(nudged_beats):
        stretched_beat = process_beat(y[:, beat[0]:beat[1]], slice_duration, fx) 
        all.append(stretched_beat)

    else:
      source_slice_list = slice_to_frames(y, slice_duration, fx=fx)
      for i, source_slice in enumerate(source_slice_list):
        if mode == 'rosa':
          all.append(source_slice)
        else:
          save(source_slice, sl_slices+str(x).zfill(4)+'-'+str(i).zfill(4)+'.wav')

    source_slice_list = []
    y = None
    show_mem()

  if mode == 'rosa':
    random.shuffle(all)
    chop_list = all
  else:
    chop_list = glob(sl_slices+'*.wav')
  all = []
  last_slices_per_beat = slices_per_beat
  last_bpm = bpm

# Concatenate
prg('Concatenating...')
input_list = []

y = 0
if crossfade == True:
  input_list1 = []
  input_list2 = []
  input_list2.append(generate_silence(slice_duration/2))
  for x in range(0, math.ceil(total_slices/2)):
    if odds(silence_amount/100):
      add = generate_silence(slice_duration)
    else:
      add = chop_list[y]
    y += 1
    input_list1.append(add)
  for x in range(math.ceil(total_slices/2), total_slices):
    if odds(silence_amount/100):
      add = generate_silence(slice_duration)
    else:
      add = chop_list[y]
    y += 1
    input_list2.append(add)
  chop_list = []

  if reverb > 0:
    input_list1.append(generate_silence(5))
    input_list2.append(generate_silence(5))

  input_list1.append(generate_silence(slice_duration/2))
  concat_audio1 = np.concatenate(input_list1, axis=1)
  concat_audio2 = np.concatenate(input_list2, axis=1)
  
  if concat_audio1.shape[1] < concat_audio2.shape[1]:
    slen = concat_audio2.shape[1]-concat_audio1.shape[1]
    tail = np.array([[0]*slen, [0]*slen], dtype=np.float32)
    concat_audio1 = np.concatenate((concat_audio1, tail), axis=1)
  elif concat_audio1.shape[1] > concat_audio2.shape[1]:
    slen = concat_audio1.shape[1]-concat_audio2.shape[1]
    tail = np.array([[0]*slen, [0]*slen], dtype=np.float32)
    concat_audio2 = np.concatenate((concat_audio2, tail), axis=1)

  concat_audio = np.add.reduce([concat_audio1, concat_audio2])
  
else:
  for x in range(total_slices):
    if odds(silence_amount/100):
      add = generate_silence(slice_duration)
    else:
      add = chop_list[y]
    y += 1
    input_list.append(add)
  chop_list = []

  if reverb > 0:
    input_list.append(generate_silence(5))

  concat_audio = np.concatenate(input_list, axis=1)

prg('Post-processing...')

timestamp = datetime.datetime.today().strftime('%Y%m%d-%H%M%S')
post_as = rnd_str(4)+'_'+timestamp+'__'+str(bpm)+'bpm.wav'
save_as = 'frankenstein_'+post_as


if reverb > 0:
  save(concat_audio, dir_tmp+post_as)
  prg('Adding reverb...')
  !sox -v {vol} "{dir_tmp}{post_as}" -r 48000 -c 2 -b 24 "{dir_tmp}{save_as}" reverb {reverb_amount} {reverb_damping} 100 100 0 0
else:
  save(concat_audio, dir_tmp+save_as)

op(c.ok, 'OK.\n ')

if save_to_drive == True:
  !cp "{dir_tmp}{save_as}" "{output_dir}{save_as}"

waveform(concat_audio)
print('\n')
test_audio(dir_tmp+save_as)


